
# Author: Vijay Hareesh Yadav Avula

## Feature extraction using Word2Vec and saving in a file
## initial features with persons as factors are also implemented



In [2]:

import json
from collections import defaultdict
all_data_rating = []
all_data_actors = defaultdict(list)
all_data_actress= defaultdict(list)
all_data_director=defaultdict(list)
all_data_moviename = []
with open('NotNullMergedMoviesCollectionJ.json') as f:
    for line in f:
        data = json.loads(line)
        if "actor" and "actress" and "director" in data["joined_movie"][0]["value"] :
            all_data_rating.append(data["avgRating"])
            all_data_moviename.append(data["movieName"])
            for any_actor in data["joined_movie"][0]["value"]["actor"]:
                all_data_actors[any_actor].append(len(all_data_rating)-1)
            for any_actress in data["joined_movie"][0]["value"]["actress"]:
                all_data_actress[any_actress].append(len(all_data_rating)-1)
            for any_director in data["joined_movie"][0]["value"]["director"]:
                all_data_director[any_director].append(len(all_data_rating)-1)
print len(all_data_actors)
print len(all_data_actress)
print len(all_data_director)
print len(all_data_rating)
print "done !"

36966
16942
2427
5157
done !


In [3]:
all_persons_list = set([])
for (key, value) in all_data_director.iteritems():
    all_persons_list.add(key)
for (key, value) in all_data_actors.iteritems():
    all_persons_list.add(key)
for (key, value) in all_data_actress.iteritems():
    all_persons_list.add(key)
all_persons = []
#all_persons_d = []
all_persons = [w.replace(' ','') for w in all_persons_list]
#all_persons = [w.replace(',','') for w in all_persons_d]


with open("persons.csv", "w") as f:
    for i in all_persons:
        f.write(i)
        f.write("\n")
print "done !"

done !


In [4]:
all_persons[:5]

[u'Robic,Gillie',
 u'St.Angelo,Robert',
 u'Donath,Ludwig',
 u'Alessandrini,Toni',
 u'Kajioka,Junichi']

In [5]:
from gensim.models import Word2Vec

In [6]:
#import word2vec
vocab = [s.encode('utf-8').split() for s in all_persons]
model = Word2Vec(vocab, size=200, window=5, min_count=1, workers=2)

In [7]:
for x in model['Maxim,Jason']:
    print x

0.00241807
0.00122965
-0.000620425
0.00236795
0.0016461
0.000989035
-0.00199643
-0.00132642
0.0002951
3.86098e-05
0.00144276
-0.00124894
0.00194166
-0.000521876
-0.000710171
-0.000399734
-0.00210399
-0.000416495
0.00104181
-0.00110435
0.0018953
-0.00038494
-0.0021013
0.000446856
-0.00249565
-0.000494312
0.00178138
-0.0018628
-0.00207624
-0.00189979
0.000519435
-0.000533901
0.000243998
-0.00232387
0.00221034
0.00169815
0.00155169
-0.00141622
0.00096997
-0.00183034
-0.00180412
-4.0303e-05
0.00202406
0.00137133
0.00107602
-0.00133461
0.00234311
-0.000645287
-0.000415618
-0.00215706
-0.0017731
0.000667345
-0.00141935
0.000241881
-0.00198426
0.002027
-0.00218871
0.00136994
0.00028066
0.00100708
0.00143525
-0.00248288
0.00179463
-0.00137036
-0.000167283
0.00241812
0.00132196
-0.00171876
0.000330973
-0.00168388
0.00151396
-0.00228123
0.000999604
-0.00161678
0.000266163
0.000477019
0.00197504
0.000337798
0.0014497
-0.000210816
0.000438798
0.000483915
-6.08624e-07
0.00122905
0.00107456
-2.98972

In [8]:
model.init_sims(replace=True)
model.save("personsw2v.model")
# model = Word2Vec.load(fname)

In [11]:
import numpy as np
movie_feature_list = defaultdict(np.ndarray)
with open('NotNullMergedMoviesCollectionJ.json') as f:
    for line in f:
        data = json.loads(line)
        if "actor" and "actress" and "director" in data["joined_movie"][0]["value"] :
            movie_name = data["movieId"]#.encode('utf-8').strip()
            movie_feature_list[movie_name] = np.zeros(200)
            for any_actor in data["joined_movie"][0]["value"]["actor"]:
                toned_actor = any_actor.encode('utf-8').strip().replace(' ','')
                movie_feature_list[movie_name] = np.add(movie_feature_list[movie_name],model[toned_actor])
            for any_actress in data["joined_movie"][0]["value"]["actress"]:
                toned_actress = any_actress.encode('utf-8').strip().replace(' ','')
                movie_feature_list[movie_name] = np.add(movie_feature_list[movie_name],model[toned_actress])
            for any_director in data["joined_movie"][0]["value"]["director"]:
                toned_director = any_director.encode('utf-8').strip().replace(' ','')
                movie_feature_list[movie_name] = np.add(movie_feature_list[movie_name],model[toned_director])
print len(movie_feature_list)
print "done !"

5157
done !


In [12]:
import pandas as pd

In [20]:
movie_features = pd.DataFrame.from_dict(movie_feature_list,orient='index')

In [21]:
movie_features.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
8192,-0.003701,-0.117948,-0.733396,-0.051670,-0.070898,-0.465265,0.033220,-0.020641,0.212669,-0.327399,...,0.060547,-0.140851,-0.061153,-0.394481,0.312745,0.209849,-0.090122,0.005389,0.390053,0.046770
16384,0.690473,0.511723,-1.706483,0.358873,1.013207,0.524214,0.790851,0.651798,0.877576,-0.307375,...,0.555326,-0.487530,-0.534022,0.736555,0.018667,-0.148552,0.282891,0.743841,0.247933,-0.059241
16386,0.092756,-0.098720,-0.108464,0.056804,-0.407288,0.181672,0.256544,-0.468049,-0.127311,0.400950,...,0.024250,-0.407595,0.102975,0.057291,0.462064,-0.196242,0.580230,-0.275305,-0.101632,-0.233053
16389,0.073690,0.230064,-0.572615,0.206769,0.489451,0.651083,-0.420491,0.323959,-0.444324,-0.409629,...,-0.052554,0.102958,-0.230363,0.482519,0.470030,0.335884,0.227489,-0.260836,0.689779,-0.058754
16390,0.185649,-0.326169,0.228072,0.209000,-0.232662,-0.322732,-0.226592,0.275909,0.351672,-0.313688,...,-0.124167,0.309823,0.169482,-0.212963,-0.367601,0.154849,0.000893,0.047971,-0.001276,0.189704


In [22]:
movie_features.to_csv('movie_features_df.csv', sep='\t')

In [24]:
print data["joined_movie"][0]["value"]["actor"]
print data["joined_movie"][0]["value"]["actress"]
print data["joined_movie"][0]["value"]["director"]

[u'Aleksandrov, Hristo', u'Anichkin, Harry', u'Atanasov, Atanas', u'Binev, Nikolai', u'Charno, Stuart', u'Clarkson, Ross W.', u'Krauss, Ron', u'Lewis, Carl', u'Pinkham, Tyrone', u'Schultz, Woody', u'Shopov, Hristo', u'Skinner, William Ladd', u'Spader, James', u'Srebrev, Atanas', u'Vallette, Franklin A.', u'Velev, Velimer', u'Vodenicharov, Kaloian', u'Dosev, Dobrin', u'Crivello, Anthony']
[u'Stanicheva, Marianne', u'Stefanson, Leslie']
[u'Krauss, Ron']


In [16]:
import numpy as np

In [35]:
all_data = pd.DataFrame({'title':all_data_moviename, 'rating':all_data_rating,
                        'actors':all_data_actors, 'actress':all_data_actress,
                        'director':all_data_director}
                       )

In [38]:
all_data.tail()

,actors,actress,director,rating,title
5152,"[Alvarez, Roy, Aquino, Roldan, Centenera, Andr...","[Amador, Zenaida, Bracci, Teda, Delgado, Maris...","[Hill, Jack]",2.914667,The Big Bird Cage
5153,"[Arkin, Alan, Bembry, Carlton, Berkeley, Xande...","[Brook, Jayne, Damon, Una, Dennehy, Elizabeth,...","[Niccol, Andrew]",2.917672,Gattaca
5154,"[Jordan, Richard, Marshall, E.G., Waterston, Sam]","[Hurt, Mary Beth, Keaton, Diane, Page, Geraldi...","[Allen, Woody]",3.645635,Interiors
5155,"[Connolly, John G., Grapey, Marc, Franco, Jame...","[Campbell, Neve, Closson, Kimber, Dodds Frank,...","[Altman, Robert]",2.837124,The Company
5156,"[Aleksandrov, Hristo, Anichkin, Harry, Atanaso...","[Stanicheva, Marianne, Stefanson, Leslie]","[Krauss, Ron]",2.498222,Alien Hunter


In [81]:
total_data = pd.DataFrame({'rating':all_data_rating})
for keysi,values in all_data_actors.iteritems():
    new_data = pd.DataFrame({keysi : pd.Series(1,index=values)})  
    total_data.append(new_data)
for keysi,values in all_data_actress.iteritems():
    new_data = pd.DataFrame({keysi : pd.Series(1,index=values)})  
    total_data.append(new_data)
for keysi,values in all_data_director.iteritems():
    new_data = pd.DataFrame({keysi : pd.Series(1,index=values)})  
    total_data.append(new_data)
#new_data.head() 

In [1]:
total_data.head()

NameError: name 'total_data' is not defined

In [ ]:
new_data = []
for keysi,values in all_data_actress.iteritems():
    new_data.append(pd.DataFrame({keysi : pd.Series(1,index=values)}))
total_data = total_data.append(new_data[:])
new_data = []
for keysi,values in all_data_actors.iteritems():
    new_data.append(pd.DataFrame({keysi : pd.Series(1,index=values)}))
total_data = total_data.append(new_data[:])

In [90]:
director_data = total_data

In [95]:
director_data.fillna(0).to_csv("director_df.csv", sep='\t', encoding='utf-8')
#director_data.to_csv("director_df.csv", sep='\t', encoding='utf-8')

In [93]:
director_data.head()

,"?kerlund, Jonas","?tomo, Katsuhiro","Aaron, Paul","Abascal, Paul","Abbass, Hiam","Abbott, Abdul Malik","Abbott, George","Abrahams, Jim","Adamson, Al","Adetuyi, Robert",...,rating,"van Heerden, Andr","von B?ky, Josef","von Cziffra, G?za","von Krusenstjerna, Patrick","von Scherler Mayer, Daisy","von Sternberg, Josef","von Theumer, Ernst R.","von Trier, Lars","von Trotta, Margarethe"
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.411335,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.119658,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.761810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.967557,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.264994,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
director_data.fillna(0)

,"?kerlund, Jonas","?tomo, Katsuhiro","Aaron, Paul","Abascal, Paul","Abbass, Hiam","Abbott, Abdul Malik","Abbott, George","Abrahams, Jim","Adamson, Al","Adetuyi, Robert",...,rating,"van Heerden, Andr","von B?ky, Josef","von Cziffra, G?za","von Krusenstjerna, Patrick","von Scherler Mayer, Daisy","von Sternberg, Josef","von Theumer, Ernst R.","von Trier, Lars","von Trotta, Margarethe"
0,0,0,0,0,0,0,0,0,0,0,...,3.411335,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,3.119658,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,3.761810,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,2.967557,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,3.264994,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,3.018237,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,3.023265,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,3.646674,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,3.677519,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,3.232432,0,0,0,0,0,0,0,0,0
